In [1]:
@file:DependsOn("ro.jf.funds:funds-notebook-client:1.0.0")
%use dataframe
%use kandy

import ro.jf.funds.client.notebook.*
import ro.jf.funds.reporting.api.model.*

val username = "Johann-32.0"

val monthlyReportFrom = YearMonthTO(2019, 1)
val monthlyReportTo = YearMonthTO(2021, 8)
val monthlyReportForecastUntil = YearMonthTO(2022, 2)

val yearlyReportFrom = 2019
val yearlyReportTo = 2020
val yearlyReportForecastUntil = 2022

val fundName = "Expenses"
val reportViewName = "Expenses report"

val REPORT_DATA_CONFIGURATION_YAML_FILE = "../../data/provision/expenses-report-data-configuration.yaml"

val client = FundsClient()

In [2]:
val user = client.ensureUserExists(username)
user

UserTO(id=f599e459-11aa-4725-ab6b-ba3e9ec7c254, username=Johann-32.0)

In [3]:
val reportView = client.createReportView(user, reportViewName, fundName, client.fromYaml(File(REPORT_DATA_CONFIGURATION_YAML_FILE)))
reportView

ReportViewTO(id=54eb0900-df06-46d2-80e2-ccc5d8e35f28, name=Expenses report, fundId=b4e70407-64bb-4bd9-b1e5-d6fdbe51f8a5, dataConfiguration=ReportDataConfigurationTO(currency=Currency(value=RON), groups=[ReportGroupTO(name=basic, filter=RecordFilterTO(labels=[basic])), ReportGroupTO(name=home, filter=RecordFilterTO(labels=[home])), ReportGroupTO(name=shopping_services, filter=RecordFilterTO(labels=[shopping_services])), ReportGroupTO(name=transport, filter=RecordFilterTO(labels=[transport])), ReportGroupTO(name=fun, filter=RecordFilterTO(labels=[fun])), ReportGroupTO(name=gifts, filter=RecordFilterTO(labels=[gifts])), ReportGroupTO(name=development, filter=RecordFilterTO(labels=[development])), ReportGroupTO(name=investment, filter=RecordFilterTO(labels=[investment]))], reports=ReportsConfigurationTO(net=NetReportConfigurationTO(enabled=false, filter=null), valueReport=ValueReportConfigurationTO(enabled=false, filter=null), groupedNet=GenericReportConfigurationTO(enabled=false), grouped

In [4]:
val monthlyReportData = client.getMonthlyReportViewData(user, reportViewName, monthlyReportFrom, monthlyReportTo, monthlyReportForecastUntil)


In [5]:
val yearlyData = client.getYearlyReportViewData(user, reportViewName, yearlyReportFrom, yearlyReportTo, yearlyReportForecastUntil)


In [6]:
import kotlinx.datetime.DateTimeUnit
import kotlinx.datetime.TimeZone
import kotlinx.datetime.atStartOfDayIn
import kotlinx.datetime.minus
import org.jetbrains.kotlinx.kandy.ir.Plot
import java.math.BigDecimal

fun plotGroupData(
    reportData: ReportDataTO,
    plotTitle: String,
    groupItemFilter: (ReportDataGroupedBudgetItemTO) -> Boolean,
): Plot {
    val dataFrame = dataFrameOf(
        "bucket" to reportData.data.map<ReportDataItemTO, LocalDate> { it.timeBucket.from },
        "spent" to reportData.data.map { bucket ->
            bucket.groupedBudget!!.filter(groupItemFilter).map { it.spent!! }.reduce { acc, value -> acc + value }
        },
        "allocated" to reportData.data.map { bucket ->
            bucket.groupedBudget!!.filter(groupItemFilter).map { it.allocated!! }.reduce { acc, value -> acc + value }
        },
        "left" to reportData.data.map { bucket ->
            bucket.groupedBudget!!.filter(groupItemFilter).map { it.left!! }.reduce { acc, value -> acc + value }
        },
        "net" to reportData.data.map { bucket ->
            bucket.groupedBudget!!.filter(groupItemFilter).map { it.allocated!! + it.spent!! }
                .reduce { acc, value -> acc + value }
        },
    )
    return dataFrame
        .plot {
            line {
                val values =
                    listOf("spent", "allocated", "left").flatMap { dataFrame.get(it).values }.map { it as BigDecimal }
                val forecastBorderMin = minOf(BigDecimal.ZERO, values.minOrNull() ?: BigDecimal.ZERO)
                val forecastBorderMax = values.maxOrNull() ?: BigDecimal.ZERO
                val forecastBorderX = when (reportData.interval.granularity) {
                    TimeGranularityTO.YEARLY -> reportData.interval.toDate.minus(183, DateTimeUnit.DAY)
                    TimeGranularityTO.MONTHLY -> reportData.interval.toDate.minus(15, DateTimeUnit.DAY)
                    TimeGranularityTO.DAILY -> reportData.interval.toDate.minus(1, DateTimeUnit.DAY)
                }

                y(listOf(forecastBorderMin, forecastBorderMax))
                x.constant(forecastBorderX.atStartOfDayIn(TimeZone.UTC).toEpochMilliseconds())
            }
            x("bucket") {
                val format = when (reportData.interval.granularity) {
                    TimeGranularityTO.YEARLY -> "%Y"
                    TimeGranularityTO.MONTHLY -> "%b %Y"
                    TimeGranularityTO.DAILY -> "%d %b %Y"
                }
                axis.breaks(reportData.data.map {
                    it.timeBucket.from.atStartOfDayIn(TimeZone.UTC).toEpochMilliseconds()
                }, format)
            }
            line {
                y.constant(0)
            }
            line {
                y("spent")
                color = Color.RED
            }
            line {
                y("allocated")
                color = Color.GREEN
            }
            area {
                y("left")
                borderLine {
                    color = Color.ORANGE
                }
            }
            layout {
                title = plotTitle
                size = 2400 to 1200
            }
        }
}

fun plotGroup(reportData: ReportDataTO, title: String, group: String): Plot =
    plotGroupData(reportData, title) { it.group == group }

fun plotGroupsTotal(reportData: ReportDataTO, title: String): Plot =
    plotGroupData(reportData, title) { true }



In [7]:
plotGroupsTotal(yearlyData, "Yearly total expenses")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="8pGGwT"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Yearly total expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12],
"left":[23927.7861693426,9310.519925596302,8213.363010267609,7116.206094938901],
"spent":[-103911.4574768,-186186.29,-145048.8737384,-145048.8737384],
"allocated":[116306.0236461426,171597.41,143951.7168230713,143951.7168230713]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12],
"format":"%Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12],
"format":"%Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12],
"format":"%Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12],
"format":"%Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"y":"y"
},
"stat":"identity",
"data":{
"y":[-186186.29,171597.41]
},
"sampling":"none",
"x":1.5935616E12,
"inherit_aes":false,
"position":"identity",
"geom":"line"
},{
"mapping":{
"x":"bucket"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"bucket",
"y":"spent"
},
"stat":"identity",
"color":"#ee6666",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"bucket",
"y":"allocated"
},
"stat":"identity",
"color":"#3ba272",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"bucket",
"y":"left"
},
"stat":"identity",
"color":"#fc8452",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"area",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"datetime",
"column":"bucket"
}]
},
"spec_id":"2"
};
 var containerDiv = document.getElementById("8pGGwT");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 2400.0,
 height: 1200.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2019 
 
 
 
 
 
 
 
 
 2020 
 
 
 
 
 
 
 
 
 2021 
 
 
 
 
 
 
 
 
 2022 
 
 
 
 
 
 
 
 
 
 
 -200,000 
 
 
 
 
 
 
 -180,000 
 
 
 
 
 
 
 -160,000 
 
 
 
 
 
 
 -140,000 
 
 
 
 
 
 
 -120,000 
 
 
 
 
 
 
 -100,000 
 
 
 
 
 
 
 -80,000 
 
 
 
 
 
 
 -60,000 
 
 
 
 
 
 
 -40,000 
 
 
 
 
 
 
 -20,000 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 20,000 
 
 
 
 
 
 
 40,000 
 
 
 
 
 
 
 60,000 
 
 
 
 
 
 
 80,000 
 
 
 
 
 
 
 100,000 
 
 
 
 
 
 
 120,000 
 
 
 
 
 
 
 140,000 
 
 
 
 
 
 
 160,000 
 
 
 
 
 
 
 180,000 
 
 
 
 
 
 
 
 
 Yearly total expenses 
 
 
 
 
 y 
 
 
 
 
 bucket

In [8]:
plotGroupsTotal(monthlyReportData, "Monthly total expenses")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="UAYnX9"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Monthly total expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12],
"left":[14014.730000000001,16718.47,22834.65,23059.5060226864,22176.880981436003,23401.9074648476,20171.2627559568,4982.650688093201,10661.1030591987,15585.063487254702,17570.519861628203,23927.786169342602,30162.070795337102,35708.0828379781,33578.8985328505,35608.7481809199,29178.334581462605,18276.602806605995,1037.026595197889,3714.9828839651846,5710.741502132383,5446.024302824387,8485.8996873483,9310.519925596294,7708.298989054593,14892.164940841094,20910.184181962803,27045.903573145504,24978.508112089912,6130.662623616814,29234.197072622115,82240.9863663068,88787.56969964014,95712.36414408458,103236.54312556607,111134.192022171,119621.43832682635,128949.21265686964],
"spent":[-4468.51,-4301.32,-9171.45,-6967.76,-9043.57,-7024.09,-11232.58,-25034.7199255,-4539.6390361,-6776.5,-9787.68,-5529.22,-4702.18,-5442.03,-35747.94,-17363.44,-18069.43,-20971.82,-27334.72,-15561.74,-8102.56,-10342.12,-8651.84,-13896.47,-15195.09,-6317.14,-7060.76,-7597.91,-16297.24,-18845.79,-16242.68,-3827.13,-11031.394166666667,-11275.463680555555,-11353.242320601852,-11578.359180652007,-11385.183279039673,-11067.69105229298],
"allocated":[6950.02,7005.06,15287.63,7192.6160226864,8165.27,8255.99,7999.52,9846.94,10218.9435174615,11703.85,11782.42,11887.54,10934.13,10999.45,33626.2,19395.19,11641.91,10066.59,10094.99,18240.71,10110.61,10074.3,11694.38,14718.95,13596.0,13500.0,13095.1,13735.9,14227.0,0.0,39343.44,56840.05,17577.9775,18200.258125,18877.42130208333,19476.008077256945,19872.429583695022,20395.465382336275]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.554076

In [9]:
plotGroup(monthlyReportData, "Basic Expenses", "basic")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="F4NASH"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Basic Expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12],
"left":[3240.03,2832.2200000000003,4070.5899999999997,3790.534615966,3967.8336632439864,3895.4738703561966,2346.1459189250713,2510.9591313598135,3681.8220874642157,4353.361495567564,4303.4383089325065,4757.400234624895,5005.254412554915,5257.7113154497165,10329.495204872102,13453.640621189717,13945.477366043473,12907.009305866577,11884.777911979414,11531.58033830936,11122.286984309922,9506.980024384253,9957.518475897625,9719.359872765663,9922.101451750428,10787.806310148551,11316.273637869997,11974.884818913764,12851.924271241913,10099.656682771132,14880.5873114548,23060.05670204045,24024.731702040448,25100.724618707118,26401.496111762674,27772.732729239524,29278.294898172782,30892.15308118381],
"spent":[-2305.04,-2509.31,-3347.92,-2293.91,-2108.18,-2382.69,-3709.61,-2493.78,-1587.87,-2253.23,-2992.86,-2517.62,-2158.1,-2165.51,-2324.86,-948.25,-1951.84,-3154.12,-3041.34,-3989.72,-2393.26,-3636.25,-1883.73,-3184.42,-2513.19,-1835.59,-2078.71,-2087.36,-1969.62,-2751.21,-3092.31,-3185.44,-2550.924166666667,-2564.062847222222,-2474.713917824074,-2523.962577642747,-2468.924459112976,-2465.235664039057],
"allocated":[2085.01,2101.5,4586.29,2013.854615966,2286.19,2311.56,2159.88,2658.69,2759.1623689134,2925.94,2945.53,2971.84,2405.49,2419.86,7397.74,4072.98,2444.77,2113.98,2019.0,3648.14,2022.12,2014.9,2338.92,2943.75,2719.2,2700.0,2619.0,2747.2,2845.4,0.0,7868.67,11368.03,3515.599166666667,3640.055763888889,3775.48541087963,3895.199195119599,3974.486628046232,4079.093847050085]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1

In [10]:
plotGroup(monthlyReportData, "Home Expenses", "home")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="MEMTQd"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Home Expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12],
"left":[5045.070000000001,6537.41,9359.8,11261.479561104,13472.287357850628,15642.791748543106,18024.236227870915,2588.9126586299517,5741.840780343509,9730.695229806393,13802.218902337278,17918.67119253153,21894.03951121245,23548.958704803135,7296.239294381097,-888.3436263231254,-8981.821438707262,-21926.48827993351,-41175.68278051021,-44186.342846379324,-43357.55803776687,-41600.3653745913,-39998.64126465259,-37226.049366687344,-41785.97101119306,-38332.574182151926,-34240.28439358882,-30759.936287092267,-31576.46144750978,-43592.63301159698,-27825.030983061522,-8985.17579310411,-6066.009126437443,-2960.4610708818873,255.4868226366306,3607.6495406150257,7007.269151758288,11071.227063830154],
"spent":[-500.0,-609.16,-1763.9,-400.0,-400.0,-467.23,-420.0,-18881.0,-423.24,-456.78,-400.0,-400.0,-400.0,-2736.62,-29698.21,-16136.26,-12866.69,-17070.89,-23488.88,-10712.07,-3563.87,-2752.85,-3681.19,-3840.81,-10690.64,-2616.15,-1842.52,-2704.57,-7215.39,-12018.77,-1917.23,-6744.06,-4965.670833333333,-5082.487569444444,-5276.624033564815,-5409.576869695216,-5540.307442169817,-5111.113062350635],
"allocated":[2085.01,2101.5,4586.29,2301.679561104,2612.92,2641.91,2799.83,3446.42,3576.6108657802,4447.46,4477.32,4517.28,4373.62,4399.82,13450.4,7952.09,4773.14,4127.3,4239.87,7661.16,4246.46,4533.6,5262.56,6623.49,6118.2,6075.0,5892.8,6181.2,6402.1,0.0,17704.56,25578.08,7884.8375,8188.035625,8492.571927083332,8761.739587673612,8939.927053313078,9175.070974422502]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,

In [11]:
plotGroup(monthlyReportData, "Transport Expenses", "transport")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="Wfbh9A"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Transport Expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12],
"left":[719.88,862.3100000000001,412.57000000000005,634.3205554324,651.1315822607493,880.7197701242434,1232.8406684680504,546.2098090369769,1032.0948955867902,1963.736753190811,638.4670051907127,-193.22128324715243,401.0398642176667,1449.6681821365094,4917.745960022526,7244.887646902154,7422.398920703277,8428.468800914097,9152.409722319047,11514.693085285096,12346.545604403122,13358.633817378768,12083.35774254141,10201.535299707726,11932.614733003986,13778.131968280579,15314.06211070171,17784.899177264364,20107.569694852922,19872.10503727512,26170.338215572276,36237.50003139628,38301.88336472962,40465.80280917406,42726.267207322206,45280.831971982705,48205.349633698235,51229.03460055673],
"spent":[-204.24,-207.85,-1214.12,-281.75,-554.64,-348.18,-288.04,-1474.32,-331.47,-238.42,-2502.33,-2020.41,-608.51,-161.2,-230.49,0.0,-1218.9,-202.81,-588.5,0.0,-444.43,-405.02,-2905.92,-3945.53,-304.89,-181.04,-413.17,0.0,-240.1,-233.81,-792.5,-158.6,-835.4175,-867.9997916666667,-906.5814409722223,-739.9698943865741,-472.8398855854552,-486.8357093842432],
"allocated":[347.5,350.28,764.38,503.5005554324,571.57,577.97,640.07,787.74,817.4484968668,1170.39,1178.23,1188.76,1202.79,1209.98,3698.87,2327.42,1397.0,1207.99,1312.37,2371.23,1314.38,1410.4,1637.18,2060.66,2039.4,2025.0,1964.2,2472.5,2560.9,0.0,7081.74,10231.25,2899.800833333333,3031.919236111111,3167.04583912037,3294.534659047068,3397.35754730099,3510.520676242739]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,

In [12]:
plotGroup(monthlyReportData, "Shopping & Services Expenses", "shopping_services")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="h6JPMB"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Shopping & Services Expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12],
"left":[538.5500000000001,888.8300000000002,1043.44,-6.146385174799844,-453.2452323587163,-2452.764753243289,-2356.29790102239,-1666.8906946487275,-1034.2419651632458,-557.1131401318248,267.98871798532804,1100.1326468817413,1606.2899864515884,2265.632452313897,3023.0001206191782,4186.539038053448,4884.738836060754,5420.324473193933,5925.120082549085,6401.418176333189,6885.768462311802,7292.409687009475,7679.561677278049,7377.168548114984,7845.487757915708,7576.5216131705765,6797.798281661961,5727.880117876205,5757.652544710277,4818.408745859494,4187.589330597688,5590.0511508313275,5524.781984164661,5411.946220275772,5256.1324760628095,5054.774253165432,4861.9970116932745,4613.9216667651035],
"spent":[-385.57,0.0,-609.77,-1481.1,-936.99,-2494.99,-463.18,0.0,-82.99,-342.57,0.0,0.0,-149.99,0.0,-1259.78,0.0,0.0,-69.0,0.0,-429.95,0.0,0.0,-77.0,-893.12,-209.0,-944.99,-1425.22,-1755.98,-682.13,-938.77,-2600.22,-1438.67,-913.7583333333333,-989.9048611111111,-1072.39693287037,-1155.346677276235,-1177.198900382587,-1257.882142081136],
"allocated":[347.5,350.28,764.38,431.5136148252,489.89,495.33,559.9,689.31,715.3536654211,819.37,824.77,832.16,656.04,659.95,2017.62,1163.71,698.54,604.0,504.75,912.04,505.53,402.9,467.72,588.79,679.8,675.0,654.8,686.8,711.3,0.0,1967.22,2842.01,848.4891666666667,877.0690972222222,916.5831886574074,953.988454378858,984.4216589104295,1009.806797152965]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1

In [13]:
plotGroup(monthlyReportData, "Fun Expenses", "fun")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="FKK0iV"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Fun Expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12],
"left":[1741.3600000000006,2222.110000000001,3277.3600000000015,2103.3741702576017,928.6796613707919,2001.6818258062196,2011.7784176972687,1015.5754985806666,281.2458657872669,811.9364469822196,1754.0827971072956,2604.997605939011,2682.28177467336,3341.2172768188216,4070.7607106934684,5040.2503315472295,5621.980756193728,6125.984782135206,6529.796329273568,6903.063341684938,6665.643885892318,7172.965510939443,7754.185942519033,8492.131634425341,9169.075905050011,9845.2724917808,9786.282769865427,9974.620090562725,10544.769166080816,10543.901434053945,9779.553106977684,10565.622235943161,10873.450569276494,11224.812930387605,11563.480488257976,11881.642842617544,12164.986226507075,12415.291559054069],
"spent":[-1031.16,-570.0,-1237.89,-2109.0,-2235.76,0.0,-790.09,-1980.8199255,-1756.0690361,-405.5,0.0,-100.0,-579.01,0.0,-1287.38,0.0,0.0,0.0,0.0,-350.0,-619.0,0.0,0.0,0.0,0.0,0.0,-703.0,-360.0,0.0,0.0,-2342.87,-1485.44,-459.1925,-445.8752083333333,-483.0314756944444,-523.2840986689815,-566.8911068913966,-614.1320324656796],
"allocated":[1042.5,1050.75,2293.14,935.0141702576,1061.46,1073.29,799.98,984.7,1021.9130824088,936.28,942.63,951.02,656.04,659.95,2017.62,969.72,582.14,503.33,403.76,729.65,404.42,503.7,584.73,735.99,679.8,675.0,654.8,549.4,569.1,0.0,1573.75,2273.56,767.0208333333333,797.2375694444444,821.6990335648148,841.4464530285494,850.2344907809285,864.4373650126724]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1

In [14]:
plotGroup(monthlyReportData, "Gifts Expenses", "gifts")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="L0s2rf"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Gifts Expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12],
"left":[2217.84,2666.44,4353.13,4814.294835414,3863.271866211872,3522.824481273921,-1158.5319309859913,-280.1641370768532,492.9016891739818,1526.0249768968847,-1306.9940373393283,-728.3256577839528,-401.56672027176876,368.3451610779152,3588.948369675146,5330.015258040724,4488.762136124175,5020.960299141104,5814.502548043844,7552.888772359778,7456.959305990382,7778.910874159979,8606.654830494012,8942.715524531677,8653.13827305254,8952.39752866613,9430.14633452287,9466.522327597379,3846.617964195579,943.0714249190804,-2634.7517853948075,8824.440748658712,8933.205748658713,9056.946165325378,9164.506616714269,9211.734605718897,9235.074093807247,9284.239372569624],
"spent":[0.0,-392.0,-147.85,-402.0,-1930.0,-1330.0,-5561.66,-204.8,-351.0,-20.0,-3892.49,-491.19,-766.57,-330.2,-141.92,-198.28,-2005.0,-475.0,-216.0,-80.0,-1082.0,-488.0,-104.0,-843.59,-1102.37,-511.87,-298.14,-512.0,-6190.0,-2903.23,-5152.0,9185.08,-833.51,-812.8025,-839.869375,-901.1918229166668,-905.9919748263889,-889.6271393952546],
"allocated":[834.0,840.6,1834.54,863.164835414,979.88,990.75,879.94,1083.12,1124.0179138545,1053.28,1060.38,1069.78,1093.4,1099.96,3362.6,1939.55,1164.18,1006.66,1009.5,1824.07,1011.06,805.9,935.55,1177.48,815.8,810.0,785.7,549.4,569.1,0.0,1573.75,2273.56,942.275,936.5429166666667,947.429826388889,948.4198119212963,929.3314629147377,938.7924181576325]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.58829

In [15]:
plotGroup(monthlyReportData, "Development Expenses", "development")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="8S30om"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Development Expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12],
"left":[512.0,709.15,317.76000000000005,461.64866968720014,-253.07791714330816,-88.8194780127966,71.09135500387563,268.0484222113727,465.43970600618314,-2243.578275057346,-1888.68183258559,-1531.8685696034686,-1025.2680335011082,-523.4502546218963,352.7088725869825,1241.7589115097512,1796.7980050444583,2300.343425288589,2906.102781543141,3997.6820163721477,4591.095296991707,1936.489763543769,2403.262283270761,1803.658412738247,1971.8518794749798,2284.609210946383,2505.9054409296573,2877.0333280233335,3446.4359185181856,3446.152310335023,4675.911876475995,6948.491290540979,7195.525457207646,7412.592471096535,7869.173402809497,8324.826078831875,8868.467311189448,9443.345312910154],
"spent":[-42.5,-13.0,-850.0,0.0,-878.0,-1.0,0.0,0.0,-7.0,-3060.0,0.0,0.0,-40.0,-48.5,-805.3,-80.65,-27.0,0.0,0.0,0.0,0.0,-3060.0,0.0,-1189.0,-375.0,-227.5,-300.0,-178.0,0.0,0.0,-345.55,0.0,-472.9208333333333,-512.3309027777779,-300.0251446759259,-325.0272400655864,-253.029510071052,-242.865302576973],
"allocated":[208.5,210.15,458.61,143.8886696872,163.36,165.18,159.92,196.96,204.4371242167,351.13,353.56,356.7,546.75,549.93,1681.35,969.72,582.14,503.33,605.74,1094.42,606.64,402.9,467.72,588.79,543.8,540.0,523.8,549.4,569.1,0.0,1573.75,2273.56,719.955,729.3979166666667,756.6060763888889,780.679916087963,796.6707424286266,817.7433042976788]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.

In [16]:
plotGroup(monthlyReportData, "Investment Expenses", "investment")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="2alZCh"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Investment Expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12],
"left":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],
"spent":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],
"allocated":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E12,1.6409952E12,1.6436736E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12,1.6330464E12,1.6357248E12,1.6383168E